# Sioux Falls

Parameter perturbation analysis with the Sioux Falls instance. The graph was took from the TransportationNetworks git repository and the demand matrix from the Liu paper and consists of 22 pairs origin-destination.

The parameters that are analyzed are:
- Budget
- Breakpoints
- Infrastructure count

In [1]:
import functools
import os
from pprint import pp
import random

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import bcnetwork as bc

## Reading data

The models and solution data is read from a directory populated elsewhere.

In [16]:
def get_solution_path(model_path):
    dirname, basename = os.path.split(model_path)
    name, _ = os.path.splitext(basename)
    
    return os.path.join(dirname, f'solution_{name}.pkl')


def get_function_name(model_name):
    """
    Return demand transfer function
    name out of model_name
    """
    fallback = 'lineal'

    name_map = dict(
        lineal=fallback,
        sad='concave down',
        happy='concave up',
        logit='logit',
    )
    
    for match_name, name in name_map.items():
        if match_name in model_name:
            return name
        
    return fallback


def get_model_and_solution(model_path):
    """
    Return model and solution objects
    """
    solution_path = get_solution_path(model_path)
    
    if not os.path.exists(solution_path):
        return None, None

    model = bc.model.Model.load(model_path)
    if os.path.exists(solution_path):
        solution = bc.solution.Solution.load(solution_path)
    else:
        solution = None

    return model, solution
    


def get_row_from_model(model_path, model, solution):
    """
    Return param information and run information.
    """
    model_basename = os.path.basename(model_path)
    model_name, _ext = os.path.splitext(model_basename)

    return {
        'name': model_name,
        'total_demand_transfered': solution.total_demand_transfered,
        'm': bc.costs.calculate_user_cost(1, model.infrastructure_count - 1),
        'infrastructure_count': model.infrastructure_count,
        'budget': model.budget,
        'budget_factor': model._budget_factor,
        'budget_used': solution.budget_used,
        'breakpoint_count': len(model.breakpoints),
        'transfer_function': get_function_name(model_name),
        'run_time_seconds': solution.run_time_seconds,
        'did_timeout': solution.did_timeout,
    }

        
def generate_runs_dataframe(working_dir):
    """
    Return a pd.DataFrame out of a directory of
    models and solutions.
    """
    rows = []
    instances = []
    
    for entry in os.scandir(working_dir):
        if entry.is_dir() or not entry.name.endswith('.pkl') or 'solution' in entry.name:
            continue
        
        model_path = os.path.join(working_dir, entry.name)
        model, solution = get_model_and_solution(model_path)
        if not solution:
            continue

        data = get_row_from_model(model_path, model, solution)
        rows.append(data)
        instances.append((data['name'], model, solution))
        

    return instances, pd.DataFrame(rows)        

In [17]:
data_dir = '../sensitivity_4'

instances, df = generate_runs_dataframe(data_dir)
df.to_csv(os.path.join(data_dir, 'summup.csv'))

In [18]:
df.sort_values(by=['total_demand_transfered'], ascending=False)

,name,total_demand_transfered,m,infrastructure_count,budget,budget_factor,budget_used,breakpoint_count,transfer_function,run_time_seconds,did_timeout
38,sioux_falls_sad_50_breakpoints_4_infras,174,0.64,4,125.6,0.4,125.000000,50,concave down,36284.653864,True
17,sioux_falls_sad_20_breakpoints_4_infras,171,0.64,4,125.6,0.4,125.000000,20,concave down,36944.404704,True
42,sioux_falls_sad_10_breakpoints_5_infras,165,0.52,5,125.6,0.4,125.000000,10,concave down,36698.395321,True
35,sioux_falls_sad_10_breakpoints_4_infras,165,0.64,4,125.6,0.4,125.000000,10,concave down,36594.183841,True
0,sioux_falls_0.8_budget_factor,154,0.64,4,251.2,0.8,251.000000,5,lineal,1535.494904,False
10,sioux_falls_sad_5_breakpoints_5_infras,153,0.52,5,125.6,0.4,125.000000,5,concave down,7392.137568,False
21,sioux_falls_sad_5_breakpoints_4_infras,153,0.64,4,125.6,0.4,125.000000,5,concave down,1727.375693,False
36,sioux_falls_0.6_budget_factor,124,0.64,4,188.4,0.6,188.000000,5,lineal,8655.344073,False
44,sioux_falls_sad_50_breakpoints_5_infras,124,0.52,5,125.6,0.4,124.000000,50,concave down,36619.866917,True
25,sioux_falls_inv_logit_50_breakpoints_4_infras,117,0.64,4,125.6,0.4,125.000000,50,logit,36028.758483,True


## Sioux falls instance representation

In [22]:
for model_name, model, solution in instances:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10),)

    draw_model = functools.partial(
        bc.draw.draw,
        model,
        solution=solution,
        width=2,
    )

    draw_model(
        odpairs=True,
        infrastructures=True,
        ax=ax1,
    )
    ax1.set_title('Infrastructures Built')

    draw_model(
        odpairs=True,
        flows=True,
        flow_scale_factor=5,
        infrastructures=False,
        ax=ax2,
    )
    ax2.set_title('Flows')
    fig.savefig(os.path.join(data_dir, f'{model_name}.png'), dpi=300)